> Goal to find out spam or not 

import 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import string
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

load dataset 

In [ ]:
data = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')
data.head()

In [ ]:
data.info()

cleaning data 

In [ ]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis= 1, inplace = True)

change name of columns for more comfortable using 

In [ ]:
data.rename(columns={'v1': 'label', 'v2': 'text'}, inplace=True)
data.head()

In [ ]:
data.groupby('label').describe()


add count column

In [ ]:
data['length'] = data['text'].apply(len)
data.head()

In [ ]:
data.length.describe()


**Visualition **

In [ ]:
data['length'].plot(bins=100, kind='hist') 

In [ ]:
data.hist(column='length', by='label', bins=50, figsize=(15,5))


add function for drop punctuation marks

In [ ]:
def text_process(mess):
    
    nopunc = [char for char in mess if char not in string.punctuation]

    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

split set of data for training

In [ ]:
from sklearn.model_selection import train_test_split

msg_train, msg_test, label_train, label_test = \
train_test_split(data['text'], data['label'], test_size=0.3)

print(len(msg_train), len(msg_test), len(msg_train) + len(msg_test))

size of test 30% of dataset  1672 sms of  5572, for learning  -  3900  of 5572


Logistic regression

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)), 
    ('tfidf', TfidfTransformer()),  
    ('classifier', LogisticRegression()),  
])
pipeline.fit(msg_train,label_train)
predictions=pipeline.predict(msg_test)
print(classification_report(predictions,label_test))
lrscore=accuracy_score(predictions,label_test)
print(lrscore)

Naive Bayes

In [ ]:
pipeline1 = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)), 
    ('tfidf', TfidfTransformer()), 
    ('classifier', MultinomialNB()), 
])
pipeline1.fit(msg_train,label_train)
predictions1 = pipeline1.predict(msg_test)
print(classification_report(predictions1,label_test))
nbscore=accuracy_score(predictions1,label_test)
print(nbscore)

result

In [ ]:
results=[lrscore,nbscore]
n=['Logis. regression','Naiva Bayse']
ndf=pd.DataFrame(n)
rdf=pd.DataFrame(results)
rdf[1]=n
print('Accuracy')
rdf

Naiva Bayse better match than Logistic regression